In [109]:
import pandas as pd
import os
from pathlib import Path
import re

In [110]:
#inputs
inputdir = input("Directory: ")
organism = input("Organism (capitalize first lettter): ")
cell_line = input("Cell line: ")
oxidant = input("Oxidant (if none: type None): ")
ox_dose = input("Oxidant dose in µM (if none, enter 0): ")
treatment = input("Secondary treatment (if none: type None): ")
treatment_dose = input("Secondary treatment dose in µM (if none, enter 0): ")
time_ox = input("Length of oxidant treatment in minutes (if none, enter 0): ")
time_tr = input("Length of secondary treatment in minutes (if none, enter 0): ")
extra = input("Extra notes? (if none: press enter)")

In [111]:
multiline_str = """The metadata for the result files of {} are the following:
The organism is {}.
The cell line is {}.
The oxidant treatment is {}.
The {} dose is {} µM.
The secondary treatment is {}.
The {} dose is {} µM.
The length of oxidant treatment is {} minutes.
The length of secondary treatment is {} minutes.
Extra notes: {}.""".format(inputdir, organism, cell_line, oxidant, oxidant, ox_dose, treatment, treatment, treatment_dose, time_ox, time_tr, extra)
print(multiline_str)

The metadata for the result files of /home/robbe/ionbot/ionbot_0.9.5/PXD018549/ionbot_files/qe2_192801_P19-UCh_06_DW29_3_Top1_NoIodo are the following:
The organism is Human.
The cell line is A549.
The oxidant treatment is None.
The None dose is 0 µM.
The secondary treatment is None.
The None dose is 0 µM.
The length of oxidant treatment is 0 minutes.
The length of secondary treatment is 0 minutes.
Extra notes: Top1 was incubated with 200 µM HNE.


In [112]:
txt = inputdir
x = re.findall(".*\/",txt)
file_name = inputdir.removeprefix(x[0])
print("The file name is: {}".format(file_name))
y = re.compile('PXD([0-9]*)') 
accession = y.findall(inputdir)[0]
print("The PRIDE accession is {}".format(accession))

The file name is: qe2_192801_P19-UCh_06_DW29_3_Top1_NoIodo
The PRIDE accession is 018549


In [113]:
os.chdir('/home/robbe/ionbot/Processed_ionbot_files')
wd = os.getcwd()
pth = os.path.join(wd, "PXD{}".format(accession))
if os.path.exists(pth):
    os.chdir(pth)
else:
    os.mkdir(pth)
    os.chdir(pth)

In [114]:
from cmath import nan

ionbot_first = pd.read_csv(Path("{}/ionbot.first.csv".format(inputdir)))
ionbot_first_proteins = pd.read_csv(Path("{}/ionbot.first.proteins.csv".format(inputdir)))
ionbot_lower = pd.read_csv(Path("{}/ionbot.lower.csv".format(inputdir)))
ionbot_modifications = pd.read_csv(Path("{}/ionbot.modifications.csv".format(inputdir)))
ionbot_coeluting_proteins = pd.read_csv(Path("{}/ionbot.coeluting.proteins.csv".format(inputdir)))
files = [ionbot_first, ionbot_first_proteins, ionbot_lower, ionbot_modifications, ionbot_coeluting_proteins]
for i in files:
    if i.all == ionbot_first.all:
        name = "ionbot_first"
    elif i.all == ionbot_first_proteins.all:
        name = "ionbot_first_proteins"
    elif i.all == ionbot_lower.all:
        name = "ionbot_lower"
    elif i.all == ionbot_modifications.all:
        name = "ionbot_modifications"
    elif i.all == ionbot_coeluting_proteins.all:
        name = "ionbot_coeluting_proteins"
    copy_df = i.copy()
    copy_df['Accession'] = accession
    copy_df['spectrum_file'] = file_name
    copy_df['Organism'] = organism
    copy_df['Cell line'] = cell_line
    if oxidant != "":
        copy_df['Oxidant treatment'] = oxidant
    else:
        copy_df['Oxidant treatment'] = nan
    copy_df['Oxidant dose (µM)'] = ox_dose
    if treatment != "":
        copy_df['Secondary treatment'] = treatment
    else: 
        copy_df['Secondary treatment'] = nan
    copy_df['Secondary treatment dose (µM)'] = treatment_dose
    copy_df['Length of oxidant treatment (mins)'] = time_ox
    copy_df['Length of Secondary treatment (mins)'] = time_tr
    if extra != "":
        copy_df['Notes'] = extra 
    else: 
        copy_df['Notes'] = nan
    if i.all in [ionbot_first.all, ionbot_first_proteins.all]:
        copy_df['First/Lower/Coeluting'] = "First"
    elif i.all == ionbot_lower.all:
        copy_df['First/Lower/Coeluting'] = "Lower"
    elif i.all == ionbot_coeluting_proteins.all:
        copy_df['First/Lower/Coeluting'] = "Coeluting"
    if i.all in [ionbot_first.all, ionbot_lower.all]:
        copy_df = copy_df[copy_df['database'] != 'D']
    copy_df.to_csv("{}{}.csv".format(file_name,name))
    
    
    
    